In [1]:
from __future__ import print_function
import IPython
import sys
import pygame
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.


# Process Music Data

Value as a note, which comprises a pitch and a duration. For example, if you press down a specific piano key for 0.5 seconds, then you have just played a note. 

In [2]:
#"R,0.125" : a rest element of  (1/32) length, or 1/8 quarter note. 
#"C,0.125<M-2,m-6>" : chord note of (1/32) length, generated
#                             anywhere from minor 6th down to major 2nd down.

In [3]:
# get data
chords, abstract_grammars = get_musical_data('./midi/original_metheny.mid')
corpus, values, val_indices, indices_val = get_corpus_data(abstract_grammars)
print('corpus length:', len(corpus))
print('total # of values:', len(values))

corpus length: 193
total # of values: 78


In [4]:
chords[1][0].offset

476.0

In [5]:
chords[1][1].offset

478.0

In [6]:
chords[2][0].offset

480.0

In [7]:
abstract_grammars

['C,0.500 C,0.333,<P1,d-5> C,0.250,<M-2,m-6> C,0.250,<M3,d-3> C,0.250,<d6,m2> C,0.250,<d5,P1> C,0.667,<M2,d-4> C,0.250,<d1,P-5> C,0.250,<P-4,d-8> S,0.250,<d4,M-2> S,0.250,<P5,A1> C,0.250,<d5,P1>',
 'C,0.500 S,0.250,<m2,P-4> C,0.250,<P4,m-2> A,0.250,<P4,m-2> S,0.500,<d1,P-5> C,0.250,<P1,d-5> C,0.250,<m2,P-4> A,0.250,<m2,P-4> C,0.250,<M2,d-4> A,0.250,<d4,M-2> C,0.250,<P4,m-2> C,0.250,<P4,m-2>',
 'C,0.250 C,0.250,<d2,A-4> A,0.250,<m-2,d-6> C,0.250,<d3,M-3> A,0.250,<M3,d-3> C,0.250,<d6,m2> C,0.250,<A4,d-2> C,0.250,<d3,M-3> C,0.250,<m2,P-4> A,0.250,<m2,P-4> S,0.250,<P1,d-5> C,0.250,<m2,P-4> C,0.250,<P1,d-5> C,0.250,<m2,P-4> A,0.250,<m2,P-4>',
 'C,0.250 S,0.250,<d6,m2> X,0.250,<A4,d-2> S,0.250,<dd5,d1> C,0.250,<P4,m-2> C,0.250,<M3,d-3> C,0.250,<dd5,d1> S,0.250,<P4,m-2> C,0.250,<P4,m-2> C,0.250,<M3,d-3> C,0.250,<dd5,d1> A,0.250,<M3,d-3>',
 'C,0.500 A,0.333,<M2,d-4> C,0.250,<d4,M-2> A,0.333,<M2,d-4> C,0.250,<m2,P-4> C,0.250,<A-4,P-8> C,0.250,<P4,m-2> A,0.250,<P4,m-2> S,0.250,<d5,P1> A,0.333,<P

In [8]:
corpus

['C,0.500',
 'C,0.333,<P1,d-5>',
 'C,0.250,<M-2,m-6>',
 'C,0.250,<M3,d-3>',
 'C,0.250,<d6,m2>',
 'C,0.250,<d5,P1>',
 'C,0.667,<M2,d-4>',
 'C,0.250,<d1,P-5>',
 'C,0.250,<P-4,d-8>',
 'S,0.250,<d4,M-2>',
 'S,0.250,<P5,A1>',
 'C,0.250,<d5,P1>',
 'C,0.500',
 'S,0.250,<m2,P-4>',
 'C,0.250,<P4,m-2>',
 'A,0.250,<P4,m-2>',
 'S,0.500,<d1,P-5>',
 'C,0.250,<P1,d-5>',
 'C,0.250,<m2,P-4>',
 'A,0.250,<m2,P-4>',
 'C,0.250,<M2,d-4>',
 'A,0.250,<d4,M-2>',
 'C,0.250,<P4,m-2>',
 'C,0.250,<P4,m-2>',
 'C,0.250',
 'C,0.250,<d2,A-4>',
 'A,0.250,<m-2,d-6>',
 'C,0.250,<d3,M-3>',
 'A,0.250,<M3,d-3>',
 'C,0.250,<d6,m2>',
 'C,0.250,<A4,d-2>',
 'C,0.250,<d3,M-3>',
 'C,0.250,<m2,P-4>',
 'A,0.250,<m2,P-4>',
 'S,0.250,<P1,d-5>',
 'C,0.250,<m2,P-4>',
 'C,0.250,<P1,d-5>',
 'C,0.250,<m2,P-4>',
 'A,0.250,<m2,P-4>',
 'C,0.250',
 'S,0.250,<d6,m2>',
 'X,0.250,<A4,d-2>',
 'S,0.250,<dd5,d1>',
 'C,0.250,<P4,m-2>',
 'C,0.250,<M3,d-3>',
 'C,0.250,<dd5,d1>',
 'S,0.250,<P4,m-2>',
 'C,0.250,<P4,m-2>',
 'C,0.250,<M3,d-3>',
 'C,0.250,

In [9]:
values

{'A,0.250,<M2,d-4>',
 'A,0.250,<M3,d-3>',
 'A,0.250,<P-4,d-8>',
 'A,0.250,<P1,d-5>',
 'A,0.250,<P4,m-2>',
 'A,0.250,<d4,M-2>',
 'A,0.250,<d5,P1>',
 'A,0.250,<m-2,d-6>',
 'A,0.250,<m2,P-4>',
 'A,0.333,<A4,d-2>',
 'A,0.333,<M2,d-4>',
 'A,0.333,<P1,d-5>',
 'C,0.250',
 'C,0.250,<A-4,P-8>',
 'C,0.250,<A4,d-2>',
 'C,0.250,<M-2,m-6>',
 'C,0.250,<M2,d-4>',
 'C,0.250,<M3,d-3>',
 'C,0.250,<P-4,d-8>',
 'C,0.250,<P1,d-5>',
 'C,0.250,<P11,M7>',
 'C,0.250,<P4,m-2>',
 'C,0.250,<P5,A1>',
 'C,0.250,<d1,P-5>',
 'C,0.250,<d2,A-4>',
 'C,0.250,<d3,M-3>',
 'C,0.250,<d4,M-2>',
 'C,0.250,<d5,P1>',
 'C,0.250,<d6,m2>',
 'C,0.250,<dd5,d1>',
 'C,0.250,<m-2,d-6>',
 'C,0.250,<m2,P-4>',
 'C,0.250,<m3,m-3>',
 'C,0.250,<m7,M3>',
 'C,0.333,<A4,d-2>',
 'C,0.333,<M2,d-4>',
 'C,0.333,<P1,d-5>',
 'C,0.333,<P5,A1>',
 'C,0.333,<d1,P-5>',
 'C,0.333,<m2,P-4>',
 'C,0.333,<m3,m-3>',
 'C,0.333,<m7,M3>',
 'C,0.500',
 'C,0.500,<P4,m-2>',
 'C,0.500,<m2,P-4>',
 'C,0.500,<m6,M2>',
 'C,0.667,<M2,d-4>',
 'C,0.667,<d6,m2>',
 'C,0.750,<m3

In [10]:
val_indices

{'A,0.250,<d5,P1>': 0,
 'S,0.750,<m3,m-3>': 1,
 'C,0.250,<d1,P-5>': 2,
 'S,0.250,<d6,m2>': 3,
 'C,0.333,<m7,M3>': 4,
 'C,0.250,<M-2,m-6>': 5,
 'C,0.250,<P1,d-5>': 6,
 'X,0.250,<A4,d-2>': 7,
 'C,0.333,<A4,d-2>': 8,
 'S,0.333': 9,
 'C,0.250,<m7,M3>': 10,
 'C,0.250,<d2,A-4>': 11,
 'A,0.250,<m-2,d-6>': 12,
 'C,0.250,<P-4,d-8>': 13,
 'C,0.667,<d6,m2>': 14,
 'C,0.250,<P11,M7>': 15,
 'S,0.333,<d1,P-5>': 16,
 'C,0.250,<A-4,P-8>': 17,
 'S,0.250,<A4,d-2>': 18,
 'S,0.250,<d4,M-2>': 19,
 'S,0.250,<P5,A1>': 20,
 'X,0.250,<M-2,m-6>': 21,
 'C,0.250,<d6,m2>': 22,
 'C,0.500': 23,
 'C,0.500,<m6,M2>': 24,
 'S,0.250,<d1,P-5>': 25,
 'A,0.333,<P1,d-5>': 26,
 'C,0.250,<P4,m-2>': 27,
 'C,0.333,<d1,P-5>': 28,
 'X,0.250,<d2,A-4>': 29,
 'A,0.250,<M2,d-4>': 30,
 'C,0.250,<P5,A1>': 31,
 'C,0.500,<P4,m-2>': 32,
 'C,0.500,<m2,P-4>': 33,
 'A,0.250,<m2,P-4>': 34,
 'A,0.250,<M3,d-3>': 35,
 'A,0.250,<d4,M-2>': 36,
 'C,0.667,<M2,d-4>': 37,
 'S,0.250,<d5,P1>': 38,
 'S,0.250,<d2,A-4>': 39,
 'C,0.333,<m3,m-3>': 40,
 'C,0.75

In [11]:
indices_val

{0: 'A,0.250,<d5,P1>',
 1: 'S,0.750,<m3,m-3>',
 2: 'C,0.250,<d1,P-5>',
 3: 'S,0.250,<d6,m2>',
 4: 'C,0.333,<m7,M3>',
 5: 'C,0.250,<M-2,m-6>',
 6: 'C,0.250,<P1,d-5>',
 7: 'X,0.250,<A4,d-2>',
 8: 'C,0.333,<A4,d-2>',
 9: 'S,0.333',
 10: 'C,0.250,<m7,M3>',
 11: 'C,0.250,<d2,A-4>',
 12: 'A,0.250,<m-2,d-6>',
 13: 'C,0.250,<P-4,d-8>',
 14: 'C,0.667,<d6,m2>',
 15: 'C,0.250,<P11,M7>',
 16: 'S,0.333,<d1,P-5>',
 17: 'C,0.250,<A-4,P-8>',
 18: 'S,0.250,<A4,d-2>',
 19: 'S,0.250,<d4,M-2>',
 20: 'S,0.250,<P5,A1>',
 21: 'X,0.250,<M-2,m-6>',
 22: 'C,0.250,<d6,m2>',
 23: 'C,0.500',
 24: 'C,0.500,<m6,M2>',
 25: 'S,0.250,<d1,P-5>',
 26: 'A,0.333,<P1,d-5>',
 27: 'C,0.250,<P4,m-2>',
 28: 'C,0.333,<d1,P-5>',
 29: 'X,0.250,<d2,A-4>',
 30: 'A,0.250,<M2,d-4>',
 31: 'C,0.250,<P5,A1>',
 32: 'C,0.500,<P4,m-2>',
 33: 'C,0.500,<m2,P-4>',
 34: 'A,0.250,<m2,P-4>',
 35: 'A,0.250,<M3,d-3>',
 36: 'A,0.250,<d4,M-2>',
 37: 'C,0.667,<M2,d-4>',
 38: 'S,0.250,<d5,P1>',
 39: 'S,0.250,<d2,A-4>',
 40: 'C,0.333,<m3,m-3>',
 41: 'C,

In [12]:
# number of different values or words in corpus
N_values = len(set(corpus))

# Create Sentences

In [13]:
# cut the corpus into semi-redundant sequences of max_len values
max_len = 20
sentences = []
next_values = []
step = 3

for i in range(0, len(corpus) - max_len, step):
    sentences.append(corpus[i: i + max_len]) #[i, i+max_len)
    next_values.append(corpus[i + max_len])
print('nb sequences:', len(sentences))

nb sequences: 58


# Transform data into binary matrices

In [14]:
X = np.zeros((len(sentences), max_len, N_values), dtype=np.bool)
y = np.zeros((len(sentences), N_values), dtype=np.bool)

In [15]:
for i, sentence in enumerate(sentences):
    for t, val in enumerate(sentence):
        X[i, t, val_indices[val]] = 1
    y[i, val_indices[next_values[i]]] = 1

In [16]:
X.shape

(58, 20, 78)

# LSTM Models

In [17]:
from __future__ import print_function

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
import numpy as np

In [18]:
N_epochs=128
# build a 2 stacked LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(max_len, N_values)))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(N_values))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.fit(X, y, batch_size=128, nb_epoch=N_epochs)

/Users/mingshi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Epoch 1/128
58/58 [==============================] - 1s 15ms/step - loss: 4.3571
Epoch 2/128
58/58 [==============================] - 0s 802us/step - loss: 4.3164
Epoch 3/128
58/58 [==============================] - 0s 797us/step - loss: 4.2653
Epoch 4/128
58/58 [==============================] - 0s 1ms/step - loss: 4.1114
Epoch 5/128
58/58 [==============================] - 0s 1ms/step - loss: 3.8075
Epoch 6/128
58/58 [==============================] - 0s 944us/step - loss: 3.8065
Epoch 7/128
58/58 [==============================] - 0s 962us/step - loss: 3.5922
Epoch 8/128
58/58 [==============================] - 0s 897us/step - loss: 3.5675
Epoch 9/128
58/58 [==============================] - 0s 926us/step - loss: 3.4632
Epoch 10/128
58/58 [==============================] - 0s 870us/step - loss: 3.5236
Epoch 11/128
58/58 [==============================] - 0s 1ms/step - loss: 3.4495
Epoch 12/128
58/58 [==============================] - 0s 968us/step - loss: 3.3024
Epoch 13/128
58/58 [

58/58 [==============================] - 0s 786us/step - loss: 0.5795
Epoch 101/128
58/58 [==============================] - 0s 809us/step - loss: 0.7056
Epoch 102/128
58/58 [==============================] - 0s 745us/step - loss: 0.6552
Epoch 103/128
58/58 [==============================] - 0s 767us/step - loss: 0.7958
Epoch 104/128
58/58 [==============================] - 0s 795us/step - loss: 1.1506
Epoch 105/128
58/58 [==============================] - 0s 801us/step - loss: 0.8212
Epoch 106/128
58/58 [==============================] - 0s 844us/step - loss: 0.5128
Epoch 107/128
58/58 [==============================] - 0s 798us/step - loss: 0.4290
Epoch 108/128
58/58 [==============================] - 0s 955us/step - loss: 0.3601
Epoch 109/128
58/58 [==============================] - 0s 737us/step - loss: 0.3685
Epoch 110/128
58/58 [==============================] - 0s 710us/step - loss: 0.3837
Epoch 111/128
58/58 [==============================] - 0s 784us/step - loss: 0.3752
Epoch 

# Predict

In [19]:
def __sample(a, temperature=1.0):
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

In [20]:
def __predict(model, x, indices_val, diversity):
    preds = model.predict(x, verbose=0)[0]
    next_index = __sample(preds, diversity)
    next_val = indices_val[next_index]
    
    return next_val

In [21]:
#def generate_sonnet(init, model, rounds=600, is_sample=True, temperature=1.0):
#    in_string = init.lower()
#    for i in range(rounds):
#        output = predict_next(string_to_x(in_string), model)
#        if is_sample:
#            n = y_to_char_sample(output, temperature)
#        else:
#            n = y_to_char(output)
#        in_string += n
#    return in_string

In [22]:
def __generate_grammar(model, corpus, abstract_grammars, values, val_indices,
                       indices_val, max_len, max_tries, diversity):
    curr_grammar = ''
    
    # init
    start_index = np.random.randint(0, len(corpus) - max_len)
    sentence = corpus[start_index: start_index + max_len] 
    
    running_length = 0.0
    while running_length <= 4.1:
        x = np.zeros((1, max_len, len(values)))
        for t, val in enumerate(sentence):
            x[0, t, val_indices[val]] = 1.
        
        next_val = __predict(model, x, indices_val, diversity)
        
        if (running_length < 0.00001):
            tries = 0
            while (next_val.split(',')[0] == 'R' or 
                len(next_val.split(',')) != 2):
                
                if tries >= max_tries:
                    print('Gave up on first note generation after', max_tries, 
                        'tries')
                    rand = np.random.randint(0, len(abstract_grammars))
                    next_val = abstract_grammars[rand].split(' ')[0]
                else:
                    next_val = __predict(model, x, indices_val, diversity)
                
                tries += 1
        
        sentence = sentence[1:]  
        sentence.append(next_val)
        
        if (running_length > 0.00001): curr_grammar += ' '
        curr_grammar += next_val
        
        length = float(next_val.split(',')[1])
        running_length += length
    return curr_grammar

In [23]:
max_len = 20
max_tries = 1000
diversity = 1

# musical settings
bpm = 130

In [24]:
# set up audio stream
out_stream = stream.Stream()



In [25]:
curr_offset = 0.0

In [26]:
import pygame

from pygame.locals import *

In [27]:
loopEnd = len(chords)

In [28]:
for loopIndex in range(1, loopEnd):
    curr_chords = stream.Voice()
    for j in chords[loopIndex]:
        curr_chords.insert((j.offset % 4), j)
        
    # generate grammar
    curr_grammar = __generate_grammar(model=model, corpus=corpus, 
                                      abstract_grammars=abstract_grammars, 
                                      values=values, val_indices=val_indices, 
                                      indices_val=indices_val, 
                                      max_len=max_len, max_tries=max_tries,
                                      diversity=diversity)
    
    print(curr_grammar)

    curr_grammar = curr_grammar.replace(' A',' C').replace(' X',' C')

    # Pruning #1: smoothing measure
    curr_grammar = prune_grammar(curr_grammar)

    # Get notes from grammar and chords
    curr_notes = unparse_grammar(curr_grammar, curr_chords)
    
    print(curr_notes)
    
    # Pruning #2: removing repeated and too close together notes
    curr_notes = prune_notes(curr_notes)

    # quality assurance: clean up notes
    curr_notes = clean_up_notes(curr_notes)

    # print # of notes in curr_notes
    print('After pruning: %s notes' % (len([i for i in curr_notes
        if isinstance(i, note.Note)])))
    
    # insert into the output stream
    for m in curr_notes:
        out_stream.insert(curr_offset + m.offset, m)
    
    for mc in curr_chords:
        out_stream.insert(curr_offset + mc.offset, mc)
        
    curr_offset += 4.0

out_stream.insert(0.0, tempo.MetronomeMark(number=bpm))

S,0.333 C,0.250,<P4,m-2> C,0.250,<M2,d-4> C,0.250,<P4,m-2> A,0.250,<m-2,d-6> A,0.250,<m-2,d-6> A,0.250,<m-2,d-6> A,0.250,<m-2,d-6> A,0.250,<m-2,d-6> C,0.250,<d6,m2> X,0.250,<d1,P-5> C,0.250,<m2,P-4> C,0.250,<m2,P-4> C,0.250,<m2,P-4> C,0.250,<m2,P-4> C,0.250,<m2,P-4> C,0.250,<m2,P-4>
<music21.stream.Voice 0xa3483e748>
After pruning: 17 notes
S,0.250 X,0.250,<d1,P-5> X,0.250,<d1,P-5> X,0.250,<d1,P-5> S,0.250 C,0.250 S,0.333,<m2,P-4> C,0.250 S,0.333,<m2,P-4> S,0.333,<m2,P-4> C,0.250,<P1,d-5> C,0.250,<P1,d-5> C,0.250,<P1,d-5> C,0.250,<P5,A1> C,0.250,<P5,A1> C,0.250,<P1,d-5>
<music21.stream.Voice 0xa3480f710>
After pruning: 16 notes
C,0.250 S,0.250,<d2,A-4> C,0.250,<d2,A-4> C,0.250,<d2,A-4> C,0.250,<d2,A-4> C,0.250,<d2,A-4> C,0.250,<d2,A-4> C,0.250,<d2,A-4> C,0.250,<d2,A-4> C,0.250,<d1,P-5> C,0.250,<d1,P-5> C,0.250,<d1,P-5> A,0.250,<d5,P1> A,0.250,<d5,P1> A,0.250,<d5,P1> A,0.250,<d5,P1> A,0.250,<d5,P1>
<music21.stream.Voice 0xa34891eb8>
After pruning: 17 notes
S,0.250 C,0.250,<m2,P-4> C,0.2

In [29]:
play = lambda x: midi.realtime.StreamPlayer(x).play()
play(out_stream)

In [35]:
out_fn = './midi/generated1.mid'

In [32]:
mf = midi.translate.streamToMidiFile(out_stream)


In [36]:
mf.open(out_fn, 'wb')
mf.write()
mf.close()